# magicpatch demos

## Still runs normal JavaScript

In [1]:
var n = "bob";
console.log("hi", n);

hi bob


## %magic example
Magicpatch adds [%magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to IJavascript. Magics are intended to make development faster and easier. Here's a magic called %echo that does... pretty much what you would expect:

In [2]:
%echo hi there

hi there


## %addmagic

You can also add your own magics. The syntax is `%addcmd cmdname fnname`.  `cmdname` must start with a symbol like `%` or `.`, and `fnname` is a string that is the name of a function. `fnname` is not checked to see if it exists until the magic is called. If you want to create a npm module for adding magics, you can use the global method `$$.addMagic()`, which has the same syntax.

In [3]:
function bar() {
    console.log("LETS ALL GO TO THE BAR");
}
%addmagic %foo bar

[ added magic: '%foo' which will call function 'bar' ]


In [4]:
%foo

LETS ALL GO TO THE BAR


In [5]:
function sayMyName(cmd, name) {
    console.log("my name is:", name);
}

In [6]:
%addmagic %name sayMyName

[ added magic: '%name' which will call function 'sayMyName' ]


In [7]:
%name mud

my name is: mud


## !exec
Lines that start with `!` will execute in the underlying shell.

In [8]:
!ls -laoF

total 488
drwxr-xr-x   8 ampower  staff     256 Dec 29 11:56 ./
drwxr-xr-x  17 ampower  staff     544 Dec 29 09:20 ../
-rw-r--r--@  1 ampower  staff    6148 Dec 29 08:58 .DS_Store
drwxr-xr-x   5 ampower  staff     160 Dec 28 14:41 .ipynb_checkpoints/
-rw-r--r--   1 ampower  staff    2419 Dec 29 11:23 Untitled.ipynb
-rw-r--r--   1 ampower  staff   10204 Dec 29 07:54 Untitled1.ipynb
-rw-r--r--   1 ampower  staff   30799 Dec 29 11:56 magicdemo.ipynb
-rw-r--r--@  1 ampower  staff  190387 Dec 29 08:56 quickDemo.png
[ process 'ls -laF' exited with code 0 ]


0

In [9]:
! ls

Untitled.ipynb
Untitled1.ipynb
magicdemo.ipynb
quickDemo.png
[ process 'ls' exited with code 0 ]


0

In [10]:
!npm audit

                                                                                
                       === npm audit security report ===                        
                                                                                
found 0 vulnerabilities
 in 359 scanned packages
[ process 'npm audit' exited with code 0 ]


0

## Var Assignment
The output of %magic and !exec commands can be assigned to a variable for future reference.

In [11]:
let mod = %require ../test/helpers/testModule.js

[ loading /Users/ampower/Projects/personal/magicpatch/test/helpers/testModule.js ]


In [12]:
console.log("Module is:", mod);

Module is: { source: 'helpers/testModule.js', worked: true }


In [13]:
let dirListing = !ls -laF

evalmachine.<anonymous>:1
let dirListing = !ls -laF
                 ^

ReferenceError: ls is not defined
    at evalmachine.<anonymous>:1:18
    at Script.runInThisContext (vm.js:132:18)
    at runInThisContext (vm.js:315:38)
    at runCode (/Users/ampower/Projects/personal/magicpatch/lib/interpreter.js:327:12)
    at magicInterpreter (/Users/ampower/Projects/personal/magicpatch/lib/interpreter.js:203:17)
    at Object.ijavascriptMonkeyPatch [as runInThisContext] (/Users/ampower/Projects/personal/magicpatch/lib/interpreter.js:67:20)
    at run ([eval]:1054:15)
    at onRunRequest ([eval]:888:18)
    at onMessage ([eval]:848:13)
    at process.emit (events.js:314:20)


##  {var} substitution

Using {var} will be replaced with the variable `var`. 

In [14]:
x = 42
%echo {x}

42


In [15]:
o = {
    beer: "yum"
}
%echo {o}

{ beer: 'yum' }


In [16]:
%echo x{x}o{o}

x42o[object Object]


## View Documentation
`?` before or after a magic will show you it's documentation

In [18]:
?%echo

__%echo Documentation:__

Usage: %echo &lt;string&gt;<br>
<br>
Write arguments to the standard output.<br>


__File:__ /Users/ampower/Projects/personal/magicpatch/lib/builtin.js

In [19]:
%echo?

__%echo Documentation:__

Usage: %echo &lt;string&gt;<br>
<br>
Write arguments to the standard output.<br>


__File:__ /Users/ampower/Projects/personal/magicpatch/lib/builtin.js

## View Code
`??` before or after a magic will show you the code for the magic.

In [20]:
??%echo

__%echo Source:__

``` js
function echo(cmd) {
    console.log(this.varSubst(this.line.substring(this.startArgs)));
}
```

__File:__ /Users/ampower/Projects/personal/magicpatch/lib/builtin.js

In [21]:
%echo??

__%echo Source:__

``` js
function echo(cmd) {
    console.log(this.varSubst(this.line.substring(this.startArgs)));
}
```

__File:__ /Users/ampower/Projects/personal/magicpatch/lib/builtin.js

## Input Caching
Previously run commands can be found in a few different ways. The last three commands are available in the global variables `_i`, `_ii`, and `_iii`. All previous commands can be found in the `In` or `_ih` arrays.

In [21]:
console.log(In[0])

var n = "bob";
console.log("hi", n);


In [22]:
console.log(In[1])

%echo hi there


In [23]:
console.log(_ih[1])

%echo hi there


In [24]:
console.log("last input:\n" + _i)
console.log("\nsecond to last input:\n" + _ii)
console.log("\ninput before that:\n" + _iii)

last input:
console.log(_ih[1])

second to last input:
console.log(In[1])

input before that:
console.log(In[0])


## Output Caching
The previous three output values are stored in `_`, `__`, and `___`.

In [25]:
21 * 2

42

In [26]:
"f" + "oo"

'foo'

In [27]:
Math.PI / 2

1.5707963267948966

In [28]:
console.log("last output:", _)
console.log("second to last output:", __)
console.log("output before that:", ___)

last output: 1.5707963267948966
second to last output: foo
output before that: 42


## Cell Magic
Cell magics do something special with the entire Jupyter cell. For example, this `%%script` cell magic turns the entire shell into a bash script.

In [29]:
%%script bash
for ((i = 0; i < 10; i++)); do
    echo "loop $i"
done

loop 0
loop 1
loop 2
loop 3
loop 4
loop 5
loop 6
loop 7
loop 8
loop 9
[ process 'bash /var/folders/6j/bk9lpc4n057831vsshxv0_j8t5f96f/T/tmp-77741-7mpSjUqnnuAq' exited with code 0 ]


0

## %automagic
When automagic is enabled, magics can be called without the leading `%`.

In [30]:
echo hi

evalmachine.<anonymous>:1
echo hi
     ^^

SyntaxError: Unexpected identifier
    at new Script (vm.js:100:7)
    at createScript (vm.js:267:10)
    at runInThisContext (vm.js:315:10)
    at runCode (/Users/ampower/Projects/personal/magicpatch/lib/interpreter.js:327:12)
    at magicInterpreter (/Users/ampower/Projects/personal/magicpatch/lib/interpreter.js:203:17)
    at Object.ijavascriptMonkeyPatch [as runInThisContext] (/Users/ampower/Projects/personal/magicpatch/lib/interpreter.js:67:20)
    at run ([eval]:1054:15)
    at onRunRequest ([eval]:888:18)
    at onMessage ([eval]:848:13)
    at process.emit (events.js:314:20)


In [31]:
%automagic on
echo hi

hi


## Useful Magics

In [38]:
%require ../test/helpers/testModule

[ loading /Users/ampower/Projects/personal/magicpatch/test/helpers/testModule ]


{ source: 'helpers/testModule.js', worked: true }

In [2]:
%lsmagic

Available line magics:
%addmagic %automagic %echo %lsmagic %quickref %require

Available cell magics:
%%script

Automagic is ON, % prefix IS NOT needed for line magics.


In [1]:
%quickref

# IJavascript + magicpatch
An enhanced Interactive JavaScript - Quick Reference Card
================================================================

%magic?, %magic??      : Get help, or more help for object (also works as ?%magic, %??magic).

Magic functions are prefixed by % or %%, and typically take their argumentswithout parentheses, quotes or even commas for convenience.  Line magics take asingle % and cell magics are prefixed with two %%.

The following magic functions are currently available:

%%script:
    Run a cell via a shell command.
%addmagic:
    Adds a new `function` as a magic named `name`.
%automagic:
    Make magic functions callable without having to type the initial %.
%echo:
    Write arguments to the standard output.
%lsmagic:
    No brief available.
%quickref:
    No brief available.
%require:
    Used to import modules, JSON, and local files.


In [34]:
%ls

UsageError: Line magic function '%ls' not found.


In [35]:
%ls -laF

UsageError: Line magic function '%ls' not found.


In [36]:
%who

UsageError: Line magic function '%who' not found.


In [37]:
%whos

UsageError: Line magic function '%whos' not found.
